In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
from sklearn.metrics import classification_report
import tensorflow_addons as tfa
from keras.utils import io_utils
import keras_nlp

In [4]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
#vocabulary = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)
train['review_text'] = train['review_text'].str.replace('[^\w\s]','')

C:\Users\Carmo\AppData\Local\Temp\ipykernel_35112\3592451880.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train['review_text'] = train['review_text'].str.replace('[^\w\s]','')


In [5]:
# len(vocabulary)

109499

In [5]:
voc = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    train['review_text'].tolist(),
    100000,
    vocabulary_output_file=None,
    lowercase=True,
    strip_accents=False,
    split=True,
    split_on_cjk=True,
    suffix_indicator="##",
    reserved_tokens=["[PAD]", "[CLS]", "[SEP]", "[UNK]", "[MASK]"],
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 2229: invalid start byte

In [7]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
            vocabulary=vocabulary,
            sequence_length=256,
            lowercase=True,
            split=True,
            strip_accents=True,
            oov_token='[UNK]',
        )

RuntimeError: Cannot find `oov_token="[UNK]"` in the vocabulary.
You can either update the vocabulary to include `"[UNK]"`, or pass a different value for the `oov_token` argument when creating the tokenizer.

In [ ]:
tk_train = tokenizer.tokenize(train['review_text'])

In [ ]:
inputs = keras.Input(shape=(1,), dtype=tf.float32)
x = keras_nlp.layers.TokenAndPositionEmbedding(
            vocabulary_size=len(vocabulary),
            sequence_length=256,
            embedding_dim=300
        )(inputs)

x = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=300, num_heads=2, dropout=0.15
)(inputs=x)

x = keras.layers.Dense(256, activation=keras.activations.linear)(x)
outputs = x
model = keras.Model(inputs=[inputs], outputs=outputs)

model.compile(optimizer=keras.optimizers.Adamax(),
                           loss=keras.losses.binary_crossentropy,
                           metrics=[keras.metrics.categorical_accuracy])

In [ ]:
chekpoint = keras.callbacks.ModelCheckpoint(f'checkpoint/transformer_encoder/', save_weights_only=True,
        monitor='val_binary_crossentropy',
        mode='max',
        save_best_only=True)

def scheduler(epoch, lr):
    if epoch <= 1:
        return lr
    else:
        return lr * tf.math.exp(-0.3)

scheduler = keras.callbacks.LearningRateScheduler(scheduler,0)

In [ ]:
model.fit(tk_train,tk_train, validation_split=0.2, batch_size=64, epochs=10, callbacks=[chekpoint, scheduler])